Описание используемых данных

Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".
Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

Fc(C,T)=TC,T>C−CT,T<C

где C,T C, T C,T — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

    для групп normal (control) и early neoplasia (treatment)
    для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.stats.multitest as smm

In [2]:
data = pd.read_csv('./gene_high_throughput_sequencing.csv', ',')

In [3]:
data

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917
5,STT5446_Breast_005_normal,normal,3.942275,4.488477,8.944837,12.581469,23.543887,8.683617,5.418139,50.525641,...,8.683617,1.314092,1.314092,1.314092,1.314092,1.314092,18.001936,1.314092,1.314092,1.314092
6,STT5451_Breast_006_normal,normal,1.084113,3.252340,8.917889,11.249037,19.179923,6.712171,12.439778,52.498406,...,9.091914,1.084113,1.084113,1.084113,1.084113,1.084113,11.659470,1.084113,1.084113,1.084113
7,STT5466_Breast_007_normal,normal,3.153900,1.647070,4.941211,11.529492,13.813151,8.235352,1.647070,44.226216,...,10.546396,1.647070,1.647070,1.647070,1.647070,1.647070,21.071346,1.647070,1.647070,1.647070
8,STT5472_Breast_008_normal,normal,2.551800,3.838876,16.255376,12.972108,20.169502,7.097877,5.333458,52.003891,...,9.274845,1.013982,1.013982,1.013982,1.013982,1.013982,11.030215,1.013982,1.013982,1.013982
9,STT5475_Breast_009_normal,normal,3.693128,1.231043,11.079385,12.453221,21.024058,6.780055,6.155214,49.174985,...,8.848678,1.231043,1.231043,1.231043,1.231043,1.231043,24.896300,1.231043,1.231043,1.231043


In [4]:
data.shape

(72, 15750)

In [5]:
normal_pats = data.loc[data.Diagnosis == 'normal']
early_pats  = data.loc[data.Diagnosis == 'early neoplasia']
cancer_pats = data.loc[data.Diagnosis == 'cancer']

markers = data.columns[2:]

In [6]:
pvals_NE = dict()
pvals_EC = dict()

for marker in markers:
    pvals_NE[marker] = stats.ttest_ind(normal_pats[marker], early_pats[marker], equal_var=False)
    pvals_EC[marker] = stats.ttest_ind(early_pats[marker], cancer_pats[marker], equal_var=False)

pvals_NE = pd.DataFrame.from_dict(pvals_NE, orient='index')
pvals_EC = pd.DataFrame.from_dict(pvals_EC, orient='index')

In [7]:
def write_ans(filename, ans):
    with open(str(filename) + '.txt', 'w') as f:
        f.write(str(ans))

In [8]:
write_ans(1, pvals_NE.loc[pvals_NE.pvalue < 0.05].shape[0])
write_ans(2, pvals_EC.loc[pvals_EC.pvalue < 0.05].shape[0])

Для этой части задания вам понадобится модуль multitest из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5

Обратите внимание, что применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)! 

In [9]:
rejected_NE, pvals_corr_NE, _, _ = smm.multipletests(pvals_NE.pvalue, alpha=0.025, method='holm')
rejected_EC, pvals_corr_EC, _, _ = smm.multipl/etests(pvals_EC.pvalue, alpha=0.025, method='holm')

In [86]:
pvals

(array([2047, 9195]),)
(array([ 2047,  6254,  9195, 11392]),)
[0.01252742 0.00133838]
[0.01252742 0.00133838]


In [83]:
pvals_NE_corr = pd.DataFrame(zip(markers, pvals_corr_NE), columns=['marker', 'pvalue'])
pvals_EC_corr = pd.DataFrame(zip(markers, pvals_corr_EC), columns=['marker', 'pvalue'])

[0.00133838 0.01252742 0.03173562 0.03498898 0.23067603 0.25429816
 0.32208017 0.34338161 0.40019178 0.47074919 0.48189056 0.50003091
 0.50017368 0.52087265 0.56163653 0.58287567 0.60624232 0.69121675
 0.69563867 0.76800225 0.82445141 0.85278623 0.99376679 1.        ]


In [85]:
NE_markers = pvals_NE_corr.loc[pvals_NE_corr.pvalue < 0.05].marker
EC_markers = pvals_EC_corr.loc[pvals_EC_corr.pvalue < 0.05].marker

In [78]:
def fold_change(C, T):
    C = float(C)
    T = float(T)
    if T < C:
        return abs(C / -T)
    else:
        return abs(T / C)

In [79]:
sign_NE_markers = []
sign_EC_markers = []

for marker in NE_markers:
    if fold_change(normal_pats[marker].mean(), early_pats[marker].mean()) > 1.5:
        sign_NE_markers.append(marker)

for marker in EC_markers:
    if fold_change(early_pats[marker].mean(), cancer_pats[marker].mean()) > 1.5:
        sign_EC_markers.append(marker)
        
print sign_NE_markers
print sign_EC_markers

[]
['TNS4', 'KCNK3', 'PRSS50', 'ADCYAP1R1', 'COL5A1']
